In [29]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import BooleanAdjective, PointerAdjective, QuantitativePointerAdjective, NodesGroupPointerAdjective, ComparisonAdjective, MaxRankAdjective, MinRankAdjective
from src.explainer.explanation import Possession, Assumption, If, ConditionalExplanation, CompositeExplanation

from src.explainer.explanation_tactics import OnlyRelevantComparisons, SkipQuantitativeExplanations, SkipConditionStatement

## Example search tree:

In [30]:
class MinMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MinMaxNode('leaf11', score=3)
leaf12 = MinMaxNode('leaf12', score=4)
leaf21 = MinMaxNode('leaf21', score=8)
leaf22 = MinMaxNode('leaf22', score=2)
leaf31 = MinMaxNode('leaf21', score=1)
leaf32 = MinMaxNode('leaf22', score=1)

child1 = MinMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
child2 = MinMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
child3 = MinMaxNode('child3', children=[leaf31, leaf32], score_child=leaf31)

root = MinMaxNode('root', maximizing_player_turn=True, children=[child1, child2, child3], score_child=child1)

## Framework definition

In [31]:
knowledgebase_ll = ArgumentationFramework(refer_to_nodes_as = 'node')

knowledgebase_ll.add_adjectives([
    
    BooleanAdjective("leaf",
        definition = "node.is_leaf"),


    QuantitativePointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = If("leaf"),
            explanation_if_true = Assumption("Leaf nodes have scores from the evaluation function"),
            explanation_if_false = CompositeExplanation(
                Assumption("Internal nodes have scores from children"),
                Possession("backpropagating child"))
        )),


    BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn"),


    PointerAdjective("backpropagating child",
        definition = "node.score_child",

        explanation = ConditionalExplanation(
            condition = If("opponent player turn"),
            explanation_if_true = CompositeExplanation(
                Assumption("We assume the opponent will do their best move."),
                Possession("backpropagating child", "worst")),
            explanation_if_false = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move."),
                Possession("backpropagating child", "best"))
        )),

    ComparisonAdjective("better", "score", ">"),
 
    NodesGroupPointerAdjective("siblings",
        definition = "node.parent.children",
        excluding = "node"),

    MaxRankAdjective("best", "better", "siblings"),

    MinRankAdjective("worst", "better", "siblings"),
])

In [32]:
print(knowledgebase_ll)

Propositions:
node is leaf
node has score = ?
node is opponent player turn
node has backpropagating child = ?
node1 is better than node2
node has siblings = ?
node is best
node is worst


Implications:
node is leaf ←
 (assumption) Definition of "leaf" is node.is_leaf
node has score = ? ←
 node is leaf ←
 (assumption) Definition of "leaf" is node.is_leaf
 ∧ (assumption) Leaf nodes have scores from the evaluation function
 || ¬(node is leaf) ←
 (assumption) Definition of "leaf" is node.is_leaf
 ∧ (assumption) Internal nodes have scores from children
 ∧ node has backpropagating child = ? ←
 node is opponent player turn ←
 (assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
 ∧ (assumption) We assume the opponent will do their best move.
 ∧ node is worst ←
 (assumption) By definition a node is "worst" if it's ¬("better") than all "siblings" ∧ Node ¬(better) than all nodes in siblings
 || ¬(node is opponent player turn) ←
 (assumption) Definition of "opponent

In [33]:
knowledgebase_hl = ArgumentationFramework(refer_to_nodes_as = 'move')

knowledgebase_hl.add_adjectives([
    
    BooleanAdjective("final move",
        definition = "node.is_leaf"),


    QuantitativePointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = If("final move"),
            explanation_if_true = Assumption("Given my evaluation of this position"),
            explanation_if_false = CompositeExplanation(
                Possession("next possible move"))
        )),


    BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn"),


    PointerAdjective("next possible move",
        definition = "node.score_child",

        explanation = ConditionalExplanation(
            condition = If("opponent player turn"),
            explanation_if_true = CompositeExplanation(
                Assumption("We assume the opponent will do their best move"),
                Possession("next possible move", "the best the opponent can do")),
            explanation_if_false = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move"),
                Possession("next possible move", "the best"))
        )),

    ComparisonAdjective("better", "score", ">"),
 
    NodesGroupPointerAdjective("possible alternative moves",
        definition = "node.parent.children",
        excluding = "node"),

    MaxRankAdjective("the best", "better", "possible alternative moves"),

    MinRankAdjective("the best the opponent can do", "better", "possible alternative moves"),
])

In [34]:
print(knowledgebase_hl)

Propositions:
move is final move
move has score = ?
move is opponent player turn
move has next possible move = ?
node1 is better than node2
move has possible alternative moves = ?
move is the best
move is the best the opponent can do


Implications:
move is final move ←
 (assumption) Definition of "final move" is node.is_leaf
move has score = ? ←
 move is final move ←
 (assumption) Definition of "final move" is node.is_leaf
 ∧ (assumption) Given my evaluation of this position
 || ¬(move is final move) ←
 (assumption) Definition of "final move" is node.is_leaf
 ∧ move has next possible move = ? ←
 move is opponent player turn ←
 (assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
 ∧ (assumption) We assume the opponent will do their best move
 ∧ move is the best the opponent can do ←
 (assumption) By definition a move is "the best the opponent can do" if it's ¬("better") than all "possible alternative moves" ∧ Node ¬(better) than all nodes in possible alt

In [35]:
explainer = ArgumentativeExplainer()
explainer.add_framework("lowlevel", knowledgebase_ll)
explainer.add_framework("highlevel", knowledgebase_hl)

## Set your explanations settings

In [36]:
explainer.set_tree_search_motivation(lambda root: root.children, "best")

In [37]:
explainer.add_explanation_tactic(OnlyRelevantComparisons(mode = "top_3"), to_adjective="best", to_framework='lowlevel')
explainer.del_explanation_tactic("OnlyRelevantComparisons", to_adjective="best", to_framework='lowlevel')


In [38]:
settings = {
            'with_framework': 'highlevel',
            'explanation_depth': 5 ,
            'assumptions_verbosity': 'verbose'
        }

explainer.configure_settings(settings)

## You can now explain nodes' properties

In [39]:
explainer.explain_adjective(child1, "the best")

child1 is the best ←
 	(assumption) By definition a move is "the best" if it's "better" than all "possible alternative moves"
	 ∧ child1 has possible alternative moves = child2, child3 ←
	 		(assumption) Definition of "possible alternative moves" is node.parent.children excluding node
	 ∧ child1 is better than child2 ←
	 		(assumption) By definition, move1 is "better" than move2 if move1 score > move2 score
			 ∧ child1 has score = 3 ←
			 			¬(child1 is final move) ←
						 				(assumption) Definition of "final move" is node.is_leaf
						 ∧ child1 has next possible move = leaf11 ←
						 				child1 is opponent player turn ←
										 					(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
										 ∧ (assumption) We assume the opponent will do their best move
										 ∧ leaf11 is the best the opponent can do ←
										 					(assumption) By definition a move is "the best the opponent can do" if it's ¬("better") than all "possible alternative mov

In [40]:
high_abstraction_settings = {
            'with_framework': 'highlevel',
            'explanation_depth': 4 ,
            'print_implicit_assumptions': False,
            'assumptions_verbosity': 'if_asked'
        }

explainer.configure_settings(high_abstraction_settings)

In [41]:
explainer.add_explanation_tactic(OnlyRelevantComparisons(mode = "top_1"), to_adjective="the best", to_framework='highlevel')
explainer.add_explanation_tactic(SkipQuantitativeExplanations(), to_framework='highlevel')
explainer.add_explanation_tactic(SkipConditionStatement(), to_framework='highlevel')

In [42]:
explainer.explain_adjective(child1, "the best", explanation_depth=2)

child1 is the best ←
 	child1 has possible alternative moves = child2 (only showing relevant 1)
	 ∧ child1 is better than child2


In [43]:
explainer.explain_adjective(child1, "the best")

child1 is the best ←
 	child1 has possible alternative moves = child2 (only showing relevant 1)
	 ∧ child1 is better than child2 ←
	 		child1 has next possible move = leaf11 ←
			 				leaf11 is the best the opponent can do
			 ∧ child2 has next possible move = leaf22 ←
			 				leaf22 is the best the opponent can do


In [44]:
settings = {
            'with_framework': 'lowlevel',
            'explanation_depth': 2 ,
            'assumptions_verbosity' : 'if_asked'
        }

explainer.configure_settings(settings)

In [45]:
explainer.explain_adjective(child1, "worst", explanation_depth = 0)

¬(child1 is worst)


In [46]:
explainer.explain_adjective(child1, "worst", explanation_depth = 1)

¬(child1 is worst) ←
 	child1 has siblings = child2, child3
	 ∧ child1 is better than child2 ∧ child1 is better than child3


In [47]:
explainer.explain_adjective(child1, "worst")

¬(child1 is worst) ←
 	child1 has siblings = child2, child3
	 ∧ child1 is better than child2 ←
	 		child1 has score = 3 ∧ child2 has score = 2
	 ∧ child1 is better than child3 ←
	 		child1 has score = 3 ∧ child3 has score = 1


In [48]:
explainer.explain_adjective(child1, "worst", explanation_depth = 3, print_depth=True)

¬(child1 is worst) ←
 	Depth 1:
	child1 has siblings = child2, child3
	 ∧ child1 is better than child2 ←
	 		Depth 2:
			child1 has score = 3 ←
			 			Depth 3:
						¬(child1 is leaf)
						 ∧ child1 has backpropagating child = leaf11
			 ∧ child2 has score = 2 ←
			 			Depth 3:
						¬(child2 is leaf)
						 ∧ child2 has backpropagating child = leaf22
	 ∧ child1 is better than child3 ←
	 		Depth 2:
			child1 has score = 3 ←
			 			Depth 3:
						¬(child1 is leaf)
						 ∧ child1 has backpropagating child = leaf11
			 ∧ child3 has score = 1 ←
			 			Depth 3:
						¬(child3 is leaf)
						 ∧ child3 has backpropagating child = leaf21


In [49]:
low_abstraction_settings = {
            'with_framework': 'lowlevel',
            'explanation_depth': 3 ,
            'print_implicit_assumptions': True,
            'assumptions_verbosity' : 'verbose'
        }

explainer.configure_settings(low_abstraction_settings)

In [50]:
explainer.explain_adjective(child1, "better", child2)

child1 is better than child2 ←
 	(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
	 ∧ child1 has score = 3 ←
	 		¬(child1 is leaf) ←
			 			(assumption) Definition of "leaf" is node.is_leaf
			 ∧ (assumption) Internal nodes have scores from children
			 ∧ child1 has backpropagating child = leaf11 ←
			 			child1 is opponent player turn
						 ∧ (assumption) We assume the opponent will do their best move. ∧ leaf11 is worst
	 ∧ child2 has score = 2 ←
	 		¬(child2 is leaf) ←
			 			(assumption) Definition of "leaf" is node.is_leaf
			 ∧ (assumption) Internal nodes have scores from children
			 ∧ child2 has backpropagating child = leaf22 ←
			 			child2 is opponent player turn
						 ∧ (assumption) We assume the opponent will do their best move. ∧ leaf22 is worst


In [51]:
explainer.explain_adjective(child1, "siblings")

child1 has siblings = child2, child3 ←
 	(assumption) Definition of "siblings" is node.parent.children excluding node


In [52]:
explainer.explain_adjective(child1, "best")

child1 is best ←
 	(assumption) By definition a node is "best" if it's "better" than all "siblings"
	 ∧ child1 has siblings = child2, child3 ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	 ∧ child1 is better than child2 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ child1 has score = 3 ←
			 			¬(child1 is leaf)
						 ∧ (assumption) Internal nodes have scores from children ∧ child1 has backpropagating child = leaf11
			 ∧ child2 has score = 2 ←
			 			¬(child2 is leaf)
						 ∧ (assumption) Internal nodes have scores from children ∧ child2 has backpropagating child = leaf22
	 ∧ child1 is better than child3 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ child1 has score = 3 ←
			 			¬(child1 is leaf)
						 ∧ (assumption) Internal nodes have scores from children ∧ child1 has backpropagating child = leaf11
			 ∧ child3 has score = 1 ←
			 			¬(child

In [53]:
explainer.explain_adjective(root, "backpropagating child")

root has backpropagating child = child1 ←
 	¬(root is opponent player turn) ←
	 		(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
	 ∧ (assumption) On our turn we take the maximum rated move.
	 ∧ child1 is best ←
	 		(assumption) By definition a node is "best" if it's "better" than all "siblings"
			 ∧ child1 has siblings = child2, child3 ←
			 			(assumption) Definition of "siblings" is node.parent.children excluding node
			 ∧ child1 is better than child2 ←
			 			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
						 ∧ child1 has score = 3 ∧ child2 has score = 2
			 ∧ child1 is better than child3 ←
			 			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
						 ∧ child1 has score = 3 ∧ child3 has score = 1


In [54]:
explainer.explain_adjective(child1, "best", explanation_depth=6, print_depth=True)

child1 is best ←
 	Depth 1:
	(assumption) By definition a node is "best" if it's "better" than all "siblings"
	 ∧ child1 has siblings = child2, child3 ←
	 		Depth 2:
			(assumption) Definition of "siblings" is node.parent.children excluding node
	 ∧ child1 is better than child2 ←
	 		Depth 2:
			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ child1 has score = 3 ←
			 			Depth 3:
						¬(child1 is leaf) ←
						 				Depth 4:
										(assumption) Definition of "leaf" is node.is_leaf
						 ∧ (assumption) Internal nodes have scores from children
						 ∧ child1 has backpropagating child = leaf11 ←
						 				Depth 4:
										child1 is opponent player turn ←
										 					Depth 5:
															(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
										 ∧ (assumption) We assume the opponent will do their best move.
										 ∧ leaf11 is worst ←
										 					Depth 5:
															(assumption) By d

In [55]:
explainer.explain_adjective(root, "score", explanation_depth = 15, print_depth=True)

root has score = 3 ←
 	Depth 1:
	¬(root is leaf) ←
	 		Depth 2:
			(assumption) Definition of "leaf" is node.is_leaf
	 ∧ (assumption) Internal nodes have scores from children
	 ∧ root has backpropagating child = child1 ←
	 		Depth 2:
			¬(root is opponent player turn) ←
			 			Depth 3:
						(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
			 ∧ (assumption) On our turn we take the maximum rated move.
			 ∧ child1 is best ←
			 			Depth 3:
						(assumption) By definition a node is "best" if it's "better" than all "siblings"
						 ∧ child1 has siblings = child2, child3 ←
						 				Depth 4:
										(assumption) Definition of "siblings" is node.parent.children excluding node
						 ∧ child1 is better than child2 ←
						 				Depth 4:
										(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
										 ∧ child1 has score = 3 ←
										 					Depth 5:
															¬(child1 is leaf) ←
															 						Dept

Track down that Not before the "Considering your definition of leaf"

In [56]:
explainer.query_explanation(root, "Why is child 1 maxoptimal?")